# Optimization of Code

## Tips and Tricks : Caching

### What is Caching?

<img src="./img/cache_bro.png"/>

- Saves the return values of some function calls, queries or HTTP requests, etc...

### Use Cases for Caching?

- What do you do when code takes too long to compute?
- What do you do when a call is repeated multiple times? (and get the same result)

### Caching Prerequisites

1. the function is deterministic and the results have the same value **every time** given the same inputs
2. the return value of the function is nondeterministic but continues to be useful and valid for a given period of time

### Candidates for Caching

1. Results from callables that query databases
2. Results that render static values: ie Web Requests, PDF Rendering, file content
    - Can improve performance and reduce network slowness
3. Results from **deterministic** calls that perform **complex** calculations
4. Global mappings that keep track of values with expiration times, like sessions and token expirations
5. Results that need to accessed often and quickly

### Implementation of Caching

- implemented many different ways
- common way is to use `dict` data structure

## Deterministic Caching

- easy and safe way of caching
- always return the same value given the same inputs

In [ ]:
def fib(n):
    """
    Returns the Nth Fibonacci Sequence Number Using Recursion
    
    :returns: Float
    """
    if n < 2:
        return 1
    else:
        return fib(n-1) + fib(n-2)

### How can `fib` get Cached?

**Fibonacci Visualized** 
<img src="./img/fib_diagram.png" width=75%/>

- As we moved down the called `fib`, the same method gets called multiple times but returns the same results
- This makes it a good candidate for caching

#### NOTE:

- Python evaluates instructions from left to right.
- This means, in this situation of the `fib` method the call to the higher value will happen before the lower n value.

### Create a Decorator to Cache

In [ ]:
def memorize_it(fn):
    """
    saves the call's argument function and result
    """
    call_cache = {}
    
    def memorized(argument):
        try:
            return call_cache[argument]
        except KeyError:
            return call_cache.setdefault(argument, fn(argument))
    return memorized

In [ ]:
def fib2(n):
    """
    Returns the Nth Fibonacci Sequence Number Using Recursion
    
    :returns: Float
    """
    if n < 2:
        return 1
    else:
        return fib(n-1) + fib(n-2)

In [ ]:
@memorize_it
def fib(n):
    """
    Returns the Nth Fibonacci Sequence Number Using Recursion
    
    :returns: Float
    """
    if n < 2:
        return 1
    else:
        return fib(n-1) + fib(n-2)

In [ ]:
%timeit fib(10)

In [ ]:
%timeit fib2(10)

### What is Going on?

<img src="./img/dude.PNG" />

- the decorator simple saves the inputs and output into the `dict` variable
- this works for this case, but not all cases
- luckily for us, the Python core library has built-in caching!

### `lru_cache` Decorator

In [ ]:
from functools import lru_cache

- If *maxsize* is set to None, the LRU features are disabled and the cache
can grow without bound.

- If *typed* is True, arguments of different types will be cached separately.
    + For example, f(3.0) and f(3) will be treated as distinct calls with distinct results.

- Arguments to the cached function must be hashable.

- View the cache statistics named tuple (hits, misses, maxsize, currsize) with f.cache_info().  Clear the cache and statistics with f.cache_clear(). Access the underlying function with `f.__wrapped__`.


In [ ]:
@lru_cache(5, True)  # saves last 5 calls sensative to input types
def fib3(n):
    """
    Returns the Nth Fibonacci Sequence Number Using Recursion
    
    :returns: Float
    """
    if n < 2:
        return 1
    else:
        return fib(n-1) + fib(n-2)

In [ ]:
fib3(10)

In [ ]:
fib3.cache_info()

In [ ]:
fib3.cache_clear()

## Non-deterministic Caching

- The trick with this caching method is to know when the cache should be cleared.
- Because each result could be different, ensure cache is long lived enough to be helpful, but short enough to prevent new data from coming in.
- These methods tend to be hard to track

#### Two Options:

1. Modify `lru_cache`
2. Use a 3rd Party Module:
    - PiPy Search: https://pypi.org/search/?q=caching
    - A github project on caching: https://github.com/tkem/cachetools

## Summary

- caching is efficient as long as the time is taken to interact with cache is **less than** the time the cached function takes to execute
- caching is normally done when communicating with external systems
    + caching has it's own costs
- If doing non-deterministic caching, ensure you do it carefully.  This will have consequences down stream.

<img src="./img/woohoo.jpg" >